# $\mathfrak{O}$-Orienting Problem - Brute Force Algorithm

The paper presents an algorithm to solve the *$\mathfrak{O}$-Orienting Problem* given an oracle for the *Decision $\mathfrak{O}$-Orienting Problem*. It is not currently known whether an efficient implementation of the oracle is possible. Below we provide an algorithm for the *$\mathfrak{O}$-Orienting Problem* using a brute force algorithm taking the place of the oracle.

We assume the order $\mathfrak{O}$ is given with a trace zero generator, $\mathbb{Z}[\sqrt{-d}]$. If your generator is not trace zero, you can translate it to either trace zero or $1/2$. In the case of $1/2$, the order $\mathbb{Z}+2\mathfrak{O}$ is also oriented and does have a trace zero generator.

**Note:** This code has not been optimized, its main purpose is for finding simple examples.

In [1]:
def get_isogeny(E, q, i):
    """
    Returns the 'i'th isogeny of degree 'q' from elliptic curve 'E'.
    Note that the base field of 'E' must be large enough for a basis of the q-torsion to exist.
    """
    P, Q = E.torsion_basis(q) # Gives error if field extension too small
    return E.isogeny(P + i*Q, algorithm="factored")

def get_automorphisms_of_order(E, ord):
    """
    Returns automorphism of order 'ord' on curve E. Returns None if such an automorphism doesn't exist.
    """
    for iso in E.isomorphisms(E):
        sf = iso.scaling_factor()
        if (sf != 1 and sf != -1 and sf**ord == -1):
            return iso
    return None

def is_endo_trace_zero(endomorphism):
    """
    Returns true if endomorphism has trace zero, false otherwise.
    """
    return endomorphism == -endomorphism.dual()

def find_endomorphisms_from_degrees(start_curve, remaining_qs, on_found_endo=None, isog_chain=[]):
    """
    Finds endomorphisms by walking in isogeny graph with a sequence of given degrees.

    Given starting curve 'start_curve',
        a list of isogenies representing a walk from the starting curve 'isog_chain' to the current curve,
        and a list of primes q, 'remaining_qs', where it remains to evaluate all possible q-isogenies,
        recursively performs a depth-first search of the isogeny graph to find endomorphisms.
    Input 'on_found_endo' provides a function that is called whenever an endomorphism is found. If it returns True, we stop.

    Example:
        evaluate_path(E, [3,3,7])
        Explore all paths of 3-isogenies from E, then from those codomains all 3-isogenies, and from those codomains all 7-isogenies, recording all endomorphisms.
    """
    # Take the next prime from the 'remaining_qs' list
    if remaining_qs == []:
        return
    q = remaining_qs[0]
    remaining_qs = remaining_qs[1:]

    # We continue walking from the codomain of the last isogeny
    cur_curve = start_curve if len(isog_chain) == 0 else isog_chain[-1].codomain()

    for i in range(0, q+1):
        # Walk to next curve
        phi = get_isogeny(cur_curve, q, i)
        new_curve = phi.codomain()
        if (new_curve.is_isomorphic(start_curve) == True):
            # We have found an endomorphism, add it to the list
            iso = new_curve.isomorphism_to(start_curve)
            endo = phi
            for j in range(0, len(isog_chain)):
                endo = endo.pre_compose(isog_chain[-j-1])
            endo = endo.post_compose(iso)
            # Call function to handle finding an endomorphism
            if on_found_endo(endo):
                return True
        # Perform next step in walk
        new_isog_chain = isog_chain.copy()
        new_isog_chain.append(phi)
        if find_endomorphisms_from_degrees(start_curve, remaining_qs, on_found_endo, new_isog_chain):
            return True

def solve_orienting_problem(E, d, find_all=True, find_suborder_orientations=True, do_print=True):
    """
    Solves the O-Orienting Problem for a curve 'E' and imaginary quadratic order in the form Z[w]=Z[sqrt{-d}], i.e. generator w has trace 0 and norm 'd'.
    Note:
    - E must be given over a large enough field extension so a basis for the q-torsion exists for each q | d.
    - If d has too many factors the algorithm will not terminate.
    """
    # Get special automorphisms - these are automorphisms which can change the trace of the resulting endomorphism
    auts = []
    if E.j_invariant() == 1728:
        auts.append(get_automorphisms_of_order(E, 2))
    if E.j_invariant() == 0:
        aut = get_automorphisms_of_order(E, 3)
        auts.append(aut, aut**2)

    # Construct the array of isogeny degrees we want in the isogeny path. Factors of d could appear multiple times depending on exponent.
    qs = factor(d)
    remaining_qs = []
    for q in qs:
        remaining_qs.extend([q[0]] * q[1])

    # Find all isogeny paths with degrees given by 'remaining_qs' array (or partial paths) which give endomorphisms
    #    The following function is called when an endomorphism is found
    valid_endomoprhisms = []
    endomorphisms_orienting_suborders = []
    def on_found_endo(endo):
        # Check trace is zero, or if is there an automorphism that makes it trace zero
        trace_zero = is_endo_trace_zero(endo)
        if not trace_zero:
            aut_trace_zero = [is_endo_trace_zero(endo.post_compose(aut)) for aut in auts]
            if True in aut_trace_zero:
                endo = endo.post_compose(auts[aut_trace_zero.index(True)])
                trace_zero = True
        if not trace_zero:
            if do_print: print("Found endomorphism of degree: " + str(endo.degree()) + " - not trace zero")
            return False
        # Check the degree is what we want
        if endo.degree() == d:
            if do_print: print("Found trace zero endomorphism of correct degree!")
            valid_endomoprhisms.append(endo)
            if not find_all: return True
        else:
            if do_print: print("Found trace zero endomorphism found of smaller degree: " + str(endo.degree()) + " so a suborder might orient the curve.")
            endomorphisms_orienting_suborders.append(endo)

    find_endomorphisms_from_degrees(E, remaining_qs, on_found_endo)

    if do_print:
        print()
        if len(valid_endomoprhisms) > 0:
            print("Success, found " + str(len(valid_endomoprhisms)) + " trace zero endomorphisms of correct degree.")
        if len(valid_endomoprhisms) == 0 and len(endomorphisms_orienting_suborders) > 0:
            print("Failed to find trace zero endomorphisms of correct degree. But, at least" + str(len(endomorphisms_orienting_suborders)) + " endomorphisms of smaller degree exist, each orienting a suborder.")
        if len(valid_endomoprhisms) == 0 and len(endomorphisms_orienting_suborders) == 0:
            print("Failed to find any orientations.")

    return valid_endomoprhisms, endomorphisms_orienting_suborders

## Example: $E_0$

Typically in isogeny-based cryptography we take $p \equiv 3 \text{ mod } 4$, and work from the starting curve $E_0: y^2 = x^3 + x$. Then the endomorphism ring is $\langle \frac{1+j}{2}, \frac{1+i}{2}, j, k \rangle_{\mathbb{Z}} \subset B_{p, \infty}$. Suppose we have $p = 78539$, element $\frac{i+j}{2}$ lies in the endomorphism ring and has reduced norm $\frac{p+1}{4} = 19635 = 3\cdot 5\cdot 7\cdot 11\cdot 17$. Hence the quadratic order $\mathbb{Z}[\sqrt{-19635}]$ should orient $E_0$.

We use our code to check this. Note this is particularity nice as all the factors of $d$ divide $p+1$ by construction, so we don't need to work over an extension field to evaluate the isogenies.

In [2]:
p = 78539
Fp2 = GF(p^2)
E = EllipticCurve(Fp2, [1,0])
d = 19635

solve_orienting_problem(E, d, find_all=False, find_suborder_orientations=True, do_print=True)

Found trace zero endomorphism of correct degree!

Success, found 1 trace zero endomorphisms of correct degree.


([Composite morphism of degree 19635 = 3*5*7*11*17:
    From: Elliptic Curve defined by y^2 = x^3 + x over Finite Field in z2 of size 78539^2
    To:   Elliptic Curve defined by y^2 = x^3 + x over Finite Field in z2 of size 78539^2],
 [])

## Example 2

To setup this example, we performed a short walk from $E_0$ to a curve $E$, found the endomorphism ring of $E$ by pushing the known endomorphism ring of $E_0$ through the isogeny. We then picked a small trace-zero element of the endomorphism ring and factorized its norm $d$. And we found the smallest field extension so each prime factor $q \mid d$ has a $q$-torsion basis of $E$ defined. These are the parameters we obtained:

We use the prime $p = 2\cdot 509 -1 = 1019$. The curve $E: y^2 = x^3 + (125z + 362)x^2 + x$ where $z = \sqrt{-1}$ in $\mathbb{F}_{p^2}$. This curve is oriented by $\mathbb{Z}[\sqrt{-d}]$ where $d = 3^4 \cdot 5 \cdot 11$ and the smallest field extension we can use to evaluate $3, 5$ and $11$-isogenies is $\mathbb{F}_p^{10}$.

Our code recovers the endomorphism defining the orientation:

In [3]:
p = 1019
d = 3^4 * 5 * 11

# We define the curve over Fp2
F2.<z> = GF(p^2, modulus=x^2+1)
E = EllipticCurve(F2, [0, 125*z + 362, 0, 1, 0])
# Then change the base field to give correct field extension
F10 = F2.extension(5)
E = E.change_ring(F10)

valid_endomoprhisms, _ = solve_orienting_problem(E, d, find_all=False, find_suborder_orientations=True, do_print=True)
endomorphism = valid_endomoprhisms[0]

print(endomorphism.degree() == d)
print(is_endo_trace_zero(endomorphism))


Found endomorphism of degree: 4455 - not trace zero
Found endomorphism of degree: 4455 - not trace zero
Found endomorphism of degree: 4455 - not trace zero
Found endomorphism of degree: 4455 - not trace zero
Found endomorphism of degree: 81 - not trace zero
Found endomorphism of degree: 4455 - not trace zero
...
Found endomorphism of degree: 4455 - not trace zero
Found endomorphism of degree: 81 - not trace zero
Found endomorphism of degree: 4455 - not trace zero
Found endomorphism of degree: 4455 - not trace zero
Found endomorphism of degree: 4455 - not trace zero
Found endomorphism of degree: 405 - not trace zero
Found trace zero endomorphism of correct degree!

Success, found 1 trace zero endomorphisms of correct degree.
True
True
